In [211]:
import math
import pygame

In [212]:
class Body:
    def __init__(self, m, r, d, a, name, immobile):
        # The following assumes the body is placed relative to coordinates (0, 0) and that the body is circular
        self.m = m # Mass of body (kg)
        self.r = r # Radius of body (m)
        self.d = d # Distance from body to (0, 0) (m)
        self.a = a # Angle of body from origin (°)
        self.name = name
        self.immobile = immobile
        self.coordinates = (d*math.cos(math.radians(a)), d*math.sin(math.radians(a))) # Current coordinates in (x, y) format.
        self.cur_vx, self.cur_vy = 0, 0 # Current velocity vectors in x and y directions
        
    
    def move_body(self, t): 
        # t is the amount of time the body travels along its velocity vectors
        if self.immobile: return
        sx = self.cur_vx*t
        sy = self.cur_vy*t
        new_coordinates = (self.coordinates[0]+sx, self.coordinates[1]+sy)
        self.d = math.dist(self.coordinates, new_coordinates)
        self.a = math.degrees(math.atan2(new_coordinates[1], new_coordinates[0]))
        self.coordinates = new_coordinates


In [213]:
class GravitySimulation:
    def __init__(self):
        self.bodies = []
        self.t = 1 # Scales velocity vectors. A higher value means faster computation but less accurate movement.
        self.G = 6.6743e-11 # Gravitational constant
        
        
    def add_body(self, m, r, d, a, name="", immobile=False):
        new_body = Body(m, r, d, a, name, immobile)
        self.bodies.append(new_body)
        
        
    def gravity(self):
        pairs = []
        for i in range(len(self.bodies)):
            for j in self.bodies[i+1:]:
                pairs.append((self.bodies[i], j))
                 
        for i in pairs:
            body1, body2 = i[0], i[1]
            f = (self.G*body1.m*body2.m)/(math.dist(body1.coordinates, body2.coordinates)**2) # Force of gravity between the bodies
            v1, v2 = f/body1.m*self.t, f/body2.m*self.t # Velocity due to gravity on each body
            # Coordinates of each body
            b1x, b1y = body1.coordinates[0], body1.coordinates[1]
            b2x, b2y = body2.coordinates[0], body2.coordinates[1]
            # Angle of velocity vectors for each body with respect to x axis
            a1, a2 = math.degrees(math.atan2(b2y-b1y, b2x-b1x)),  math.degrees(math.atan2(b1y-b2y, b1x-b2x))
            # Velocity due to gravity on each body in x and y directions
            v1x, v1y = v1*math.cos(math.radians(a1)), v1*math.sin(math.radians(a1))
            v2x, v2y = v2*math.cos(math.radians(a2)), v2*math.sin(math.radians(a2))
            # Increment each body's vx and vy
            body1.cur_vx += v1x
            body1.cur_vy += v1y
            body2.cur_vx += v2x
            body2.cur_vy += v2y
            
        for i in self.bodies:
            i.move_body(self.t)
            
            
    def generate_stable_orbit(self, body, orbit_body, l):
        # l is the length of the semimajor axis of orbit
        # The orbit body is what the body orbits around
        # This assumes the orbit body does not move
        # This also assumes that nothing else interacts with the two bodies
        d = math.dist(body.coordinates, orbit_body.coordinates)
        vo = math.sqrt(self.G*orbit_body.m*(2/d-1/l)) # Orbital velocity
        # Coordinates of the two bodies
        bx, by = body.coordinates[0], body.coordinates[1]
        obx, oby = orbit_body.coordinates[0], orbit_body.coordinates[1]
        a = math.degrees(math.atan2(oby-by, obx-bx)) # Angle of body from orbit body
        vox = vo*math.cos(math.radians(-90-a))
        voy = -vo*math.sin(math.radians(-90-a))
        
        f = (self.G*body.m*orbit_body.m)/(math.dist(body.coordinates, orbit_body.coordinates)**2) # Force of gravity between the bodies
        vg = f/body.m*self.t  # Velocity due to gravity on the body
        # Velocity due to gravity on the body in x and y directions
        vgx, vgy = vg*math.cos(math.radians(a)), vg*math.sin(math.radians(a))

        body.cur_vx, body.cur_vy = vox-vgx, voy-vgy 

In [214]:
class GUI:
    def __init__(self):
        pygame.init()
        self.vps = 60 # Stands for vectors calculated per second.
        self.sim = GravitySimulation()
        self.window = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)
        self.width, self.height = self.window.get_size()
        self.font = pygame.font.SysFont("Monospace", 20)
        self.clock = pygame.time.Clock()
        self.body_scaling = 1
        self.grid_zoom = 1
        self.pause_button_rect = pygame.rect.Rect(0, 0, 150, 150)
        self.add_button_rect = pygame.rect.Rect(0, 150, 150, 150)
        self.vps_button_rect = pygame.rect.Rect(0, 300, 150, 150)
        self.time_button_rect = pygame.rect.Rect(0, 450, 150, 150)
        self.paused = False
        self.show_vps_slider = False
        self.show_time_slider = False
        self.show_add_menu = False
        self.x_selection = "0"
        self.y_selection = "0"
        self.mass_selection = "100"
        self.radius_selection = "10"
        self.is_immobile_selection = "False"
        self.vps_slider_center = [175, 400]
        self.time_slider_center = [175, 550]
        self.gui_mainloop()
        
        
    def gui_mainloop(self):
        self.sim.add_body(10**13, 10, 100, 0)
        self.sim.add_body(10**13, 10, 200, 180)
        self.sim.add_body(0.1, 3, 110, 0)
        
        self.sim.bodies[0].cur_vy += 0.7
        self.sim.bodies[1].cur_vy -= 0.7
        self.sim.generate_stable_orbit(self.sim.bodies[2], self.sim.bodies[0], 10)
        

                        
        while True:
            self.window.fill("black")
            self.check_events()    
            self.draw_grid()
            self.draw_bodies()
            self.draw_buttons()
            if self.show_vps_slider:
                self.draw_vps_slider()
            if self.show_time_slider:
                self.draw_time_slider()
            if self.show_add_menu:
                self.draw_add_menu()
            
            if not self.paused:
                self.sim.gravity()

            pygame.display.update()
            self.clock.tick(self.vps) 
            
        
    def check_events(self):
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
                pygame.quit()
            elif event.type == pygame.MOUSEWHEEL:
                self.body_scaling *= 1+event.y*0.1
                self.grid_zoom *= 1+event.y*0.1
                if (self.grid_zoom-0.625)*(self.grid_zoom-1.6) > 0:
                    self.grid_zoom = 1
            elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                if self.pause_button_rect.collidepoint(event.pos):
                    self.paused = not self.paused
                elif self.add_button_rect.collidepoint(event.pos):
                    self.show_add_menu = not self.show_add_menu
                elif self.vps_button_rect.collidepoint(event.pos):
                    self.show_vps_slider = not self.show_vps_slider
                elif self.time_button_rect.collidepoint(event.pos):
                   self.show_time_slider = not self.show_time_slider
            elif event.type == pygame.MOUSEMOTION:
                if math.dist(event.pos, self.vps_slider_center) <= 10:
                    if event.pos[1] < 400:
                        self.vps = min(60000, 60*(-12.4875*(event.pos[1]-400)+1))
                        self.vps_slider_center[1] = (((self.vps/60)-1)/-12.4875)+400
                    else:
                        self.vps = max(1, 60*(-1/30*(event.pos[1]-400)+1))
                        self.vps_slider_center[1] = (((self.vps/60)-1)/(-1/30))+400
                if math.dist(event.pos, self.time_slider_center) <= 10:
                    if event.pos[1] < 550:
                        self.sim.t = min(100, -1.2375*(event.pos[1]-550)+1)
                        self.time_slider_center[1] = ((self.sim.t-1)/-1.2375)+550
                    else:
                        self.sim.t = max(1/60, -1/30*(event.pos[1]-550)+1)
                        self.time_slider_center[1] = (self.sim.t-1)/(-1/30)+550
                        
        
    def draw_grid(self):
        square_length = int(self.grid_zoom*min(self.width, self.height)/50)
        x_offset = lambda square_length: self.width//2%square_length
        y_offset = lambda square_length: self.height//2%square_length
        
        # Draw the grid squares
        for h in range(y_offset(square_length), self.height, square_length):
            pygame.draw.line(self.window, "#191919", (0, h), (self.width, h))
        for w in range(x_offset(square_length), self.width, square_length):
            pygame.draw.line(self.window, "#191919", (w, 0), (w, self.height))
        for h in range(y_offset(square_length*5), self.height, square_length*5):
            pygame.draw.line(self.window, "#3F3F3F", (0, h), (self.width, h))
        for w in range(x_offset(square_length*5), self.width, square_length*5):
            pygame.draw.line(self.window, "#3F3F3F", (w, 0), (w, self.height))

        # Draw lighter lines in the center
        pygame.draw.line(self.window, "#7F7F7F", (0, self.height//2), (self.width, self.height//2))
        pygame.draw.line(self.window, "#7F7F7F", (self.width//2, 0), (self.width//2, self.height))
        
        # Draw the border lines
        pygame.draw.rect(self.window, "#7F7F7F", pygame.Rect(0, 0, self.width, self.height), 1, border_radius=10)
        
        
    def draw_bodies(self):
        for i in self.sim.bodies:
            x, y = i.coordinates[0], i.coordinates[1]
            x, y = x*self.body_scaling+self.width//2, y*self.body_scaling+self.height//2
            radius = i.r*self.body_scaling+1
            pygame.draw.circle(self.window, "blue", (x, y), radius)


    def draw_buttons(self):
        # Note: for user intuition, i will be illustrating vps as time and sim.time as the length of the vectors
        # Draw pause button
        pygame.draw.rect(self.window, "white", self.pause_button_rect, width=5, border_radius=5)
        pygame.draw.rect(self.window, "white", (30, 25, 25, 100))
        pygame.draw.rect(self.window, "white", (100, 25, 25, 100))
        
        # Draw add button
        pygame.draw.rect(self.window, "white", self.add_button_rect, width=5, border_radius=5)
        pygame.draw.rect(self.window, "white", (68, 175, 20, 100))
        pygame.draw.rect(self.window, "white", (27, 215, 100, 20))
        
        # Draw vps button
        pygame.draw.rect(self.window, "white", self.vps_button_rect, width=5, border_radius=5)
        pygame.draw.circle(self.window, "white", (75, 375), 60, 5)
        pygame.draw.line(self.window, "white", (75, 375), (125, 375), width=5)
        pygame.draw.line(self.window, "white", (75, 375), (75, 330), width=5)
        
        # Draw time button
        pygame.draw.rect(self.window, "white", self.time_button_rect, width=5, border_radius=5)
        pygame.draw.line(self.window, "white", (30, 570), (115, 485), width=10)
        pygame.draw.line(self.window, "white", (70, 485), (120, 485), width=10)
        pygame.draw.line(self.window, "white", (115, 530), (115, 485), width=10)\
            
    
    def draw_vps_slider(self):
        pygame.draw.rect(self.window, "white", (150, 300, 50, 150), width=5, border_radius=5)
        pygame.draw.rect(self.window, "white", (172, 313, 7, 125), border_radius=5)
        pygame.draw.circle(self.window, "white", self.vps_slider_center, radius=10)
        cur_vps = self.font.render(f"{self.vps/60:.2f}x", 0, "white")
        self.window.blit(cur_vps, (210, 370))
        
    
    def draw_time_slider(self):
        pygame.draw.rect(self.window, "white", (150, 450, 50, 150), width=5, border_radius=5)
        pygame.draw.rect(self.window, "white", (172, 463, 7, 125), border_radius=5)
        pygame.draw.circle(self.window, "white", self.time_slider_center, radius=10)
        cur_t = self.font.render(f"{self.sim.t:.2f}x", 0, "white")
        self.window.blit(cur_t, (210, 520))
               
               
    def draw_add_menu(self):
        pygame.draw.rect(self.window, "white", (150, 0, 300, 300), 5, 5)
        x_text = self.font.render("X:", 0, "white")
        x_selection_text = self.font.render(self.x_selection, 0, "white")
        y_text = self.font.render("Y:", 0, "white")
        y_selection_text = self.font.render(self.y_selection, 0, "white")
        mass_text = self.font.render("Mass:", 0, "white")
        mass_selection_text = self.font.render(self.mass_selection, 0, "white")
        radius_text = self.font.render("Radius:", 0, "white")
        radius_selection_text = self.font.render(self.radius_selection, 0, "white")
        is_immobile = self.font.render("Is immobile:", 0, "white")
        is_immobile_selection_text = self.font.render(self.is_immobile_selection, 0, "white")
        
        # Accept input
        # Show a preview

        self.window.blit(x_text, (160, 10))
        self.window.blit(x_selection_text, (180, 10))
        self.window.blit(y_text, (160, 30))
        self.window.blit(y_selection_text, (180, 30))
        self.window.blit(mass_text, (160, 50))
        self.window.blit(mass_selection_text, (220, 50))
        self.window.blit(radius_text, (160, 70))
        self.window.blit(radius_selection_text, (250, 70))
        self.window.blit(is_immobile, (160, 90))
        self.window.blit(is_immobile_selection_text, (310, 90))
        
        # Translate x and y to radius and distance
        
        

In [215]:
GUI()

error: display Surface quit